In [4]:
import os
import sys
from importnb import Notebook
import pandas as pd
from typing import Optional, Union, List, Dict, Callable, Set, Any
from io import BytesIO
import struct

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)

In [25]:
class SimpleType:
    
    '''
    Read the next 4 bytes, in the pattern as defined by each class, from field stream
    And create a instance using that value
    '''
    
    Local = 'field'
    Remote = None
    IsComplex = False
    
    def __init__(self):
        self.streams = dict()
        
    @classmethod
    def new(cls, value:any):
        instance = cls()
        instance.value = value
        return instance
        
    def read_local(self):
        return struct.unpack(self.Pattern, self.streams[self.Local].read(self.PatternSize))
    
    def read_remote(self):
        return struct.unpack(self.Pattern, self.streams[self.Remote].read(self.PatternSize))
    
    def read_offset(self):
        return struct.unpack("I", self.streams[self.Local].read(4))[0]
        
    def unpack(self, streams:Dict):
        self.streams = streams
        self.value = self.read_local()[0]
        return self
    
    def pack(self):
        return struct.pack(self.Pattern, self.value)
    
    def write(self, stream:Dict):
        return self.pack()
    

class ComplexType(SimpleType):
    
    '''
    Read the next 4 bytes, in the pattern as defined by each class, from field stream
    which is the offset in the Remote stream
    Then read the Pattern from Remote stream and create instance
    '''
    
    Remote = 'data'
    IsComplex = True
    
    def unpack(self, streams:Dict):
        self.streams = streams
        self.streams[self.Remote].seek(self.read_offset())
        self.value = self.read_remote()[0]
        return self
    
    def write(self, streams:Dict):
        offset = len(streams[self.Remote])
        streams[self.Remote] += self.pack()
        return struct.pack("I", offset)
        

class StringType(ComplexType):
    
    def unpack(self, streams:Dict):
        self.streams = streams
        self.streams[self.Remote].seek(self.read_offset())
        length = self.read_remote()[0]
        if length == 0:
            self.value = ''
        else:
            self.value = struct.unpack("%ds" % length, self.streams[self.Remote].read(length))[0].decode('latin')
        return self
    
    def pack(self):
        length = len(self.value)
        return struct.pack(self.Pattern + "%ds" % length, length, self.value.encode('latin'))
    
    
    
class NWByte(SimpleType):
    
    ID = 0
    DType = 'byte'
    Pattern = 'B3x'
    PatternSize = struct.calcsize(Pattern)
    
    
class NWChar(SimpleType):
    
    ID = 1
    DType= 'char'
    Pattern = 'c3x'
    PatternSize = struct.calcsize(Pattern)
    
    
class NWWord(SimpleType):
    
    ID = 2
    DType = 'word'
    Pattern = 'H2x'
    PatternSize = struct.calcsize(Pattern)
    
    
class NWShort(SimpleType):
    
    ID = 3
    DType = 'short'
    Pattern = 'h2x'
    PatternSize = struct.calcsize(Pattern)
    
    
class NWDword(SimpleType):
    
    ID = 4
    DType = 'dword'
    Pattern = 'I'
    PatternSize = struct.calcsize(Pattern)
    
    
class NWInt(SimpleType):
    
    ID = 5
    DType = 'int'
    Pattern = 'i'
    PatternSize = struct.calcsize(Pattern)
    
    
class NWDword64(ComplexType):
    
    ID = 6
    DType = 'dword64'
    Pattern = 'Q'
    PatternSize = struct.calcsize(Pattern)
    
    
class NWInt64(ComplexType):
    
    ID = 7
    DType = 'int64'
    DefaultStream = 'data'
    Complex = True
    Pattern = 'q'
    PatternSize = struct.calcsize(Pattern)
    

class NWFloat(SimpleType):
    
    ID = 8
    DType = 'float'
    Pattern = 'f'
    PatternSize = struct.calcsize(Pattern)
    
    
class NWDouble(ComplexType):
    
    ID = 9
    DType = 'double'
    Pattern = 'd'
    PatternSize = struct.calcsize(Pattern)
    
    
class NWString(StringType):
    
    ID = 10
    DType = 'cexostr'
    DefaultStream = 'data'
    Complex = True
    Pattern = 'I'
    PatternSize = struct.calcsize(Pattern)
    
    
class NWResref(StringType):
    
    ID = 11
    DType = 'resref'
    Pattern = 'B'
    PatternSize = struct.calcsize(Pattern)
      

class NWLocalizedString(ComplexType):
    
    ID = 12
    DType = 'cexolocstr'
    Pattern = 'IiI'
    PatternSize = struct.calcsize(Pattern)
        
    def unpack(self, streams:Dict):
        self.streams = streams
        self.streams[self.Remote].seek(self.read_offset())
        length, strref, count = self.read_remote()
        result = []
        if count > 0:
            for substr in range(count):
                id, length = struct.unpack('2I', self.streams[self.Remote].read(8))
                data = struct.unpack("%ds" % length, self.streams[self.Remote].read(length))[0].decode('latin')
                result.append((id, data))
        self.value = (strref, result)
        return self

    def pack(self):
        if len(self.value[1]) == 0:
            return struct.pack('IiI', 8, self.value[0], 0)
        content = struct.pack('iI', self.value[0], len(self.value[1]))
        for substr in self.value[1]:
            length = len(substr[1])
            content += struct.pack("2I%ds" % length, substr[0], length, substr[1].encode('latin'))
        return struct.pack('I', len(content)) + content

    
class NWVoid(ComplexType):
    
    ID = 13
    DType = 'void'
    Pattern = 'I'
    PatternSize = struct.calcsize(Pattern)
    
    def unpack(self, streams:Dict):
        self.streams = streams
        self.streams[self.Local].seek(self.read_offset())
        length = self.read_remote()[0]
        self.value =  self.streams[self.Remote].read(length)
        return self
    
    def pack(self):
        length = len(self.value)
        pattern = "I%dB" % length
        return struct.pack(pattern, struct.calcsize(pattern), *self.value)
    
    
class NWStruct(ComplexType):
    ID = 14
    DType = 'struct'
    Local = 'struct'
    Indice = 'indice'
    Label = 'label'
    Field = 'field'
    Pattern = '3I'
    PatternSize = struct.calcsize(Pattern)
    
    def __init__(self, index:int):
        self.value = 'struct'
        self.streams = dict()
        self.index= index
        self.type = None
        self.data = None
        self.count = None
        self.fieldIndice = list()
        self.fields = list()
        
    @classmethod
    def new(cls, type:int, index:int, fields:List):
        instance = cls(index)
        instance.type = type
        instance.fields = fields
        for _ in instance.fields:
            _.parent = instance
        return instance
    
    def like(self, index:int, values:List):
        fields = [f.like(None, v) for f, v in zip(self.fields, values)]
        instance = self.new(self.type, index, fields)
        return instance
    
    def unpack(self, streams:Dict):
        self.streams = streams
        self.type, self.data, self.count = self.read_local()
        if self.count == 1:
            self.fieldIndice = [self.data]
        else:
            pattern = "%dI" % self.count
            self.streams[self.Indice].seek(self.data)
            fieldIndice = self.streams[self.Indice].read(struct.calcsize(pattern))
            self.fieldIndice = list(struct.unpack(pattern, fieldIndice))
        return self
            
    def pack(self, streams:Dict, labels:List):
        fieldOffset = len(streams[self.Field]) // 12
        indiceOffset = len(streams[self.Indice]) 
        count = len(self.fields)
        [_.pack(streams, labels) for _ in self.fields]
        if count == 1:
            data = fieldOffset
        else:
            data = indiceOffset
            indice = [fieldOffset + i for i in range(count)]
            indice = struct.pack("%dI" % count, *indice)
            streams[self.Indice] += indice
        pack = struct.pack(self.Pattern, self.type, data, count)
        streams[self.Local] += pack
        
    def write(self, streams:Dict):
        return struct.pack("I", self.index)
            
    def bind_fields(self, fields:List):
        self.fields = [fields[_] for _ in self.fieldIndice]
        for _ in self.fields:
            _.parent = self
            
    def summary(self):
        return pd.DataFrame([_.summary() for _ in self.fields])
        
    
class NWList(ComplexType):
    ID = 15
    DType = 'list'
    Remote = 'list'
    Pattern = 'I'
    PatternSize = struct.calcsize(Pattern)
    
    def unpack(self, streams:Dict):
        self.streams = streams
        self.streams[self.Remote].seek(self.read_offset())
        count = self.read_remote()[0]
        pattern = "%dI" % count
        self.value = struct.unpack(pattern, self.streams[self.Remote].read(struct.calcsize(pattern)))
        return self
    
    def pack(self):
        count = len(self.value)
        pattern = "I%dI" % count
        return struct.pack(pattern, count, *self.value)
 

class NWField(ComplexType):
    
    ID = 16
    DType = 'field'
    Remote = 'label'
    Pattern = '2I'
    PatternSize = struct.calcsize(Pattern)
    Classes = [NWByte, NWChar, NWWord, NWShort, NWDword, NWInt, NWDword64, NWInt64, NWFloat, NWDouble, NWString, NWResref, NWLocalizedString, NWVoid, NWStruct, NWList]  
    
    def __init__(self, index:int):
        self.streams = dict()
        self.structs = list()
        self.labels = list()
        self.index = index
        self.typeIndex = None
        self.labelIndex = None
        self.label = ''
        self.content = None
        self.parent = None
        
    @classmethod
    def new(cls, index:int, typeIndex:int, label:str, value:any):
        instance = cls(index)
        instance.typeIndex = typeIndex
        instance.label = label
        instance.content = cls.Classes[typeIndex].new(value)
        return instance
    
    def like(self, index:int, value:any):
        instance = self.new(index, self.typeIndex, self.label, value)
        return instance
    
    def unpack(self, streams:Dict, structs:List, labels:List):
        self.streams = streams
        self.structs = structs
        self.labels = labels
        self.typeIndex, self.labelIndex = self.read_local()
        self.label = self.labels[self.labelIndex]
        if self.typeIndex == NWStruct.ID:
            self.content = self.structs[self.read_offset()]
        else:
            self.content = self.Classes[self.typeIndex]().unpack(self.streams)
        return self
            
    def pack(self, streams:Dict, labels:List):
        if self.label in labels:
            labelIndex = labels.index(self.label)
        else:
            labelIndex = len(labels)
            labels.append(self.label)
        streams[self.Local] += struct.pack(self.Pattern, self.content.ID, labelIndex) + self.content.write(streams)
        
    def summary(self):
        if self.typeIndex == NWStruct.ID:
            value = f'Struct {self.content.index}'
        else:
            value = self.content.value
        return pd.Series([self.index, self, self.parent.index, self.parent, self.parent.type, self.label, self.content.DType, value]
                         ,index=['Field Index', 'Field', 'StructIndex', 'Struct', 'StructType', 'Label', 'Type', 'Value'])